In [ ]:
import parsl
from parsl import *

In [ ]:
local_config = {
    "sites" : [
        { "site" : "Threads",
          "auth" : { "channel" : None },
          "execution" : {
              "executor" : "threads",
              "provider" : None,
              "max_workers" : 4
          }
        }],
    "globals" : {"lazyErrors" : True}
}

dfk = DataFlowKernel(config=local_config)

In [ ]:
## Define Apps ##
@App('bash', dfk)
def Combine(inputs=[], outputs=[], stdout='stdout.txt', stderr='stderr.txt'):
    return 'perl ./perl/Combine.pl ' + ' '.join(inputs) + ' ' + str(outputs[0])

@App('bash', dfk)
def SingleChannel(inputs=[], outputs=[], channel='1'):
    #return 'perl ./perl/SingleChannel.pl %s %s %s' % (inputs[0], outputs[0], channel)
    return 'perl ./perl/SingleChannel.pl %s "'.format(inputs[0]) + ' '.join(outputs) + '" "' + ' '.join(channel) + '"'

@App('bash', dfk)
def Frequency(inputs=[], outputs=[], column, binType, binValue)
    return 'perl ./perl/Frequency.pl %s %s %s %s %s' % (inputs[0], outputs[0], column, binType, binValue)

In [ ]:
## Analysis Parameters ##

# For Combine
thresholdAll = ['files/6119.2016.0104.1.thresh', 'files/6203.2016.0104.1.thresh']
combineOut = ['combineOut']

# For SingleChannel
singlechannel_channel = ['1']
singleChannelOut = ['singleChannelOut']

# For Frequency
freq_col = '5'
freq_binType = '1'
freq_binValue = '2'
freqOut = ['freqOut']

In [ ]:
## Workflow ##

# 1) Combine() takes the input threshold files and combines
#    them into a single file with name given by `combineOut`
Combine_future = Combine(inputs=thresholdAll, outputs=combineOut)

# 2) SingleChannel() takes the combined data file and divides it into channels
SingleChannel_future = SingleChannel(inputs=Combine_future.outputs, outputs=singleChannelOut, 
                                     channel=singlechannel_channel)

# 3) We want to apply the Frequency data transformation to (potentially) multiple files output from
#    SingleChannel
#    We define an array to hold the futures of each transformation, then fill it
Freq_futures = []
for i in range(len(SingleChannel_future.outputs)):
    Freq_futures.append(Frequency(inputs=[SingleChannel_future.outputs[i]], outputs=freqOut, 
                                  column=freq_col, binType=freq_binType, binValue=freq_binValue))

# Now define proper outputs from the futures
Frequency_outputs = [i.outputs[0] for i in Freq_futures]

# Wait for the final result before exiting.
x = [FreqOut.result() for FreqOut inFrequency_outputs]

print("Call to EventSearch completed with exit code:", x)